<a href="https://colab.research.google.com/github/dswh/lil_nlp_with_tensorflow/blob/main/02_03_begin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projecting embeddings on TensorFlow projector

This notebook explains how you can write the vectors of an embeddings into a TSV file to visualise it in a 3D space on the [TensorFlow projector](https://projector.tensorflow.org/)

In [ ]:
##import the required libraries and APIs
!pip install google-colab
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

Processing /root/.cache/pip/wheels/e4/e7/91/b2736701bca00e273fdc79f80e2727b558e0903d81b758eb69/google_colab-1.0.0-py2.py3-none-any.whl
  Using cached pandas-0.24.2.tar.gz (11.8 MB)
Processing /root/.cache/pip/wheels/20/fe/26/0c03adccd0cec2744261ec6b8380fbe5e726d1ea4cf3a9370e/tornado-4.5.3-cp38-cp38-linux_x86_64.whl
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
Processing /root/.cache/pip/wheels/b9/10/77/cad2fb4f8ab57cc7cecb92b9b1b0548ac932b8f81774003683/portpicker-1.2.0-py3-none-any.whl
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached requests-2.21.0-py2.py3-none-any.whl (57 kB)
  Using cached six-1.12.0-py2.py3-none-any.whl (10 kB)
  Using cached pytz-2022.2.1-py2.py3-none-any.whl (500 kB)
  Using cached prompt_toolkit-1.0.18-py3-none-any.whl (245 kB)
  Using cached simplegeneric-0.8.1.zip (12 kB)
  Using cached urllib3-1.24.3-py2.py

## Downloading the TensorFlow `imdb_review` dataset

> Make sure tensorflow_datasets is installed

In [2]:
##load the imdb reviews dataset
data, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

## Segregating training and testing sets

In [3]:
##segregate training and test set
train_data, test_data = data['train'], data['test']

##create empty list to store sentences and labels
train_sentences = []
test_sentences = []

train_labels = []
test_labels = []

In [4]:
##iterate over the train data to extract sentences and labels
for sent, label in train_data:
    train_sentences.append(str(sent.numpy().decode('utf8')))
    train_labels.append(label.numpy())

##iterate over the test set to extract sentences and labels
for sent, label in test_data:
    test_sentences.append(str(sent.numpy().decode('utf8')))
    test_labels.append(label.numpy())



In [5]:
##convert lists into numpy array
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

## Data preparation - setting up the tokenizer

In [6]:
##define the parameters for the tokenizing and padding
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


In [7]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

##training sequences and labels
train_seqs = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_seqs,maxlen=max_length, truncating=trunc_type)

##testing sequences and labels
test_seqs = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_seqs,maxlen=max_length)

In [8]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(train_sentences[1])
print(train_padded[1])
print(decode_review(train_padded[1]))

I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.
[   0    0    0    0    0    0    0    0   11   26   75  571    6  805
 2354  313  106   19   12    7  629  686    6    4 2219    5  181  584
   64 1454  110 2263    3 3951   21    2    1    3  258   41 4677    4
  174  188   21   12 4078   11 1578 2354   86    2   20   14 1907    2
  112  940   14 1811 1340  548    3  355  181  466    6  591   19   17
   55 1817    5   49   14 4

## Define the Neural Network with Embedding layer

1. Use the Sequential API.
2. Add an embedding input layer of input size equal to vocabulary size.
3. Add a flatten layer, and two dense layers.

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

##compile the model with loss function, optimizer and metrics
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 6)                 11526     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


## Model Training

In [10]:
num_epochs = 10

##train the model with training and validation set
model.fit(
    train_padded, 
    train_labels, 
    epochs=num_epochs, 
    validation_data=(test_padded, test_labels)
    )

Epoch 1/10
782/782 [==============================] - 10s 12ms/step - loss: 0.4946 - accuracy: 0.7445 - val_loss: 0.3405 - val_accuracy: 0.8522
Epoch 2/10
782/782 [==============================] - 8s 11ms/step - loss: 0.2435 - accuracy: 0.9057 - val_loss: 0.3567 - val_accuracy: 0.8433
Epoch 3/10
782/782 [==============================] - 8s 10ms/step - loss: 0.0983 - accuracy: 0.9714 - val_loss: 0.4474 - val_accuracy: 0.8263
Epoch 4/10
782/782 [==============================] - 8s 10ms/step - loss: 0.0243 - accuracy: 0.9964 - val_loss: 0.5271 - val_accuracy: 0.8296
Epoch 5/10
782/782 [==============================] - 8s 10ms/step - loss: 0.0072 - accuracy: 0.9994 - val_loss: 0.5858 - val_accuracy: 0.8291
Epoch 6/10
782/782 [==============================] - 8s 10ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.6357 - val_accuracy: 0.8296
Epoch 7/10
782/782 [==============================] - 8s 10ms/step - loss: 9.5167e-04 - accuracy: 1.0000 - val_loss: 0.6803 - val_accuracy: 0

## Deriving weights from the embedding layer

In [11]:
##isolating the first embedding layer
l1 = model.layers[0]

##extracting learned weights
weights = l1.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)
print(weights[0])

(10000, 16)
[ 0.02626998  0.00399183  0.01572668  0.04453173  0.0325235   0.00802202
 -0.0063911  -0.01890559  0.04883098 -0.02788557  0.03595058 -0.02518951
 -0.02180645  0.00809073  0.04548626 -0.05867545]


## Downloading the vectors and metadata

In [13]:
##import I/O module in python
import io

##open the text stream for vectors
vectors = io.open('vectors.tsv', 'w', encoding='utf-8')

##open the text stream for metadata
meta = io.open('meta.tsv', 'w', encoding='utf-8')


##write each word and its corresponding embedding
for index in range(1, vocab_size):
  word = reverse_word_index[index]  # flipping the key-value in word_index
  embeddings = weights[index]
  meta.write(word + "\n")
  vectors.write('\t'.join([str(x) for x in embeddings]) + "\n")

##close the stream
vectors.close()
meta.close()

In [21]:
##download the written files to your local machine
!pip install google.colab
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vectors.tsv')
  files.download('meta.tsv')

Processing /root/.cache/pip/wheels/e4/e7/91/b2736701bca00e273fdc79f80e2727b558e0903d81b758eb69/google_colab-1.0.0-py2.py3-none-any.whl
  Using cached tornado-4.5.3.tar.gz (484 kB)
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
  Using cached portpicker-1.2.0.tar.gz (17 kB)
  Using cached requests-2.21.0-py2.py3-none-any.whl (57 kB)
  Using cached six-1.12.0-py2.py3-none-any.whl (10 kB)
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached pandas-0.24.2.tar.gz (11.8 MB)
  Using cached simplegeneric-0.8.1.zip (12 kB)
  Using cached prompt_toolkit-1.0.18-py3-none-any.whl (245 kB)
  Using cached urllib3-1.24.3-py2.py3-none-any.whl (118 kB)
  Using cached idna-2.8-py2.py3-none-any.whl (58 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached pytz-2022.2.1-py2.py3-none-any.whl (500 kB)
  Created wheel for tornado: filename

  Created wheel for portpicker: filename=portpicker-1.2.0-py3-none-any.whl size=13384 sha256=012f3a63e9af6ea97f31224c37158ceb4dae2a8e9d504df6103a00b6eef8891e
  Stored in directory: /root/.cache/pip/wheels/b9/10/77/cad2fb4f8ab57cc7cecb92b9b1b0548ac932b8f81774003683
anceled
ERROR: Operation cancelled by user
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [20]:
from google.colab import files

ModuleNotFoundError: No module named 'google.colab'